In [1]:
import os



SCALA_VERSION = '2.11'
SPARK_VERSION = '2.4.7'

os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'


import findspark
findspark.init()

import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaInvoiceStream").getOrCreate()

In [3]:
spark

In [4]:


# read from kafka, here spark is consumer for kafka topic called invoices
# spark streaming works as dataframe/sql
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", "invoices2")\
  .option("group.id","invoice-group")\
  .load()

In [5]:
# print kafka stream for topic
kafkaDf.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
invoiceJsonRawDf = kafkaDf.selectExpr("timestamp", "CAST(value AS STRING)")
invoiceJsonRawDf.printSchema() # we get only value as string


root
 |-- timestamp: timestamp (nullable = true)
 |-- value: string (nullable = true)



In [7]:

import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, DoubleType, DateType
# json is object, spark DF needs schema 

schema = StructType(
        [
            StructField("InvoiceNo", IntegerType(), True),
            StructField("StockCode", StringType(), True),
            StructField("Quantity", IntegerType(), True),
            StructField("Description", StringType(), True),
            StructField("InvoiceDate", StringType(), True),
            #StructField("InvoiceDate", DateType(), True),
            StructField("UnitPrice", DoubleType(), True),
            StructField("CustomerID", IntegerType(), True),
            StructField("Country", StringType(), True),
        ]
)


In [8]:
jsonDf = invoiceJsonRawDf.withColumn("value", F.from_json("value", schema))
jsonDf.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- InvoiceNo: integer (nullable = true)
 |    |-- StockCode: string (nullable = true)
 |    |-- Quantity: integer (nullable = true)
 |    |-- Description: string (nullable = true)
 |    |-- InvoiceDate: string (nullable = true)
 |    |-- UnitPrice: double (nullable = true)
 |    |-- CustomerID: integer (nullable = true)
 |    |-- Country: string (nullable = true)



In [9]:
# now we will extract value which struct type ewith all schema field mention, to specific columns
#InvoiceNo, StockCode, ....
invoiceDf = jsonDf.select("timestamp", F.col("value.*"))
invoiceDf.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Description: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [10]:
# Give me how much money generated selling goods for last 60 seconds/1 minutes
invoiceDf = invoiceDf.withColumn("Amount", F.col("Quantity") * F.col("UnitPrice") )
invoiceDf.printSchema() 

root
 |-- timestamp: timestamp (nullable = true)
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Description: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Amount: double (nullable = true)



In [11]:

windowedAmountSum = invoiceDf.groupBy("Country", F.window(invoiceDf.timestamp, 
                                              "60 seconds", 
                                               "60 seconds"))\
                              .agg(F.sum("Amount").alias("TotalAmount"))\
                              .selectExpr("to_json(struct(*)) AS value")\
                              .selectExpr("CAST(value AS STRING)")
windowedAmountSum.printSchema()

root
 |-- value: string (nullable = true)



In [12]:

echoOnconsole = windowedAmountSum\
                .writeStream\
                .outputMode("complete")\
                .option("truncate", False)\
                .format("console")\
                .start() # start the query. spark will subscribe for data


In [13]:

windowedAmountSum \
    .writeStream \
    .format("kafka") \
    .outputMode("complete") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "aggregated-invoices") \
    .option("checkpointLocation", "file:///c:/spark/temp56") \
    .start()